<a href="https://colab.research.google.com/github/anantagr/Advance-House-pricing-prediction/blob/main/Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# 📌 Install necessary libraries (Run this first in Google Colab)
!pip install yfinance pandas matplotlib --quiet

# ✅ Import Libraries
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML, display

stock_symbols = ["ALLY"]  # Example tickers

# ✅ Function to Fetch Stock Data from Yahoo Finance
def fetch_stock_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    last_sale = info.get("previousClose")
    fifty_two_week_high = info.get("fiftyTwoWeekHigh")
    fifty_two_week_low = info.get("fiftyTwoWeekLow")
    market_cap = info.get("marketCap")
    pe_ratio = info.get("trailingPE")

    return {
        "Company": info.get("shortName", ticker),
        "Ticker": ticker,
        "Bid": f"${info.get('bid'):.2f}" if info.get('bid') is not None else None,  # Added $ and formatting
        "Ask": f"${info.get('ask'):.2f}" if info.get('ask') is not None else None,  # Added $ and formatting
        "Last Sale": f"${last_sale:.2f}" if last_sale is not None else None,
        "Open": f"${info.get('open'):.2f}" if info.get('open') is not None else None,
        "High": f"${info.get('dayHigh'):.2f}" if info.get('dayHigh') is not None else None,
        "Low": f"${info.get('dayLow'):.2f}" if info.get('dayLow') is not None else None,

        "Market Cap (B)": f"${round(market_cap / 1e9, 2)}B" if market_cap is not None else None,
        "P/E Ratio": f"{pe_ratio:.2f}" if pe_ratio is not None else None,
        "Volume": info.get("volume"),
        "Avg. Volume": info.get("averageVolume"),
        "Exchange": info.get("exchange"),
        "Yield (%)": f"{round(info.get('dividendYield', 0) * 100, 2)}%" if info.get('dividendYield') is not None else None,
        # "Margin Requirement": "30.00%",
        "52W High": f"${fifty_two_week_high:.2f}" if fifty_two_week_high is not None else None,
        "52W Low": f"${fifty_two_week_low:.2f}" if fifty_two_week_low is not None else None
    }

# ✅ Fetch Data for All Stocks
stock_data = [fetch_stock_data(ticker) for ticker in stock_symbols]

# ✅ Convert Data to Pandas DataFrame
df = pd.DataFrame(stock_data)

# ✅ Calculate % from 52-Week High (Trend Strength) and add the % sign to the column (and handle NaN values)
df["52W % from High"] = ((pd.to_numeric(df["Last Sale"].str.replace('$', ''), errors='coerce') - pd.to_numeric(df["52W Low"].str.replace('$', ''), errors='coerce')) / (pd.to_numeric(df["52W High"].str.replace('$', ''), errors='coerce') - pd.to_numeric(df["52W Low"].str.replace('$', ''), errors='coerce'))) * 100
df["52W % from High"] = df["52W % from High"].apply(lambda x: f"{x:.2f}%" if pd.notna(x) else None)


# ✅ Sort by P/E Ratio (Lower is Usually Better)
df_sorted = df.sort_values(by=["P/E Ratio"], ascending=True)

# Display Results in Table Format (Left-Aligned)
print("\n📊 Stock Comparison Based on Key Metrics:\n")

# Apply CSS styling for left alignment:
display(HTML(df_sorted.to_html(classes='table-bordered', justify='left'))) # justify='left' is the key change




📊 Stock Comparison Based on Key Metrics:



,Company,Ticker,Bid,Ask,Last Sale,Open,High,Low,Market Cap (B),P/E Ratio,Volume,Avg. Volume,Exchange,Yield (%),52W High,52W Low,52W % from High
0,Ally Financial Inc.,ALLY,$37.83,$38.70,$38.25,$38.35,$38.87,$38.25,$11.7B,21.28,1950029,3295095,NYQ,3.14%,$45.46,$31.95,46.63%


In [23]:
# 📌 Install necessary libraries (Run this first in Google Colab)
!pip install yfinance pandas matplotlib --quiet

# ✅ Import Libraries
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML, display
from datetime import date, timedelta

stock_symbols = ["ALLY"]  # Example tickers

# ✅ Function to Fetch Stock Data from Yahoo Finance
def fetch_stock_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    # Get the current price (using 'regularMarketPrice' if available, otherwise 'previousClose')
    current_price = info.get('regularMarketPrice')
    if current_price is None:
        current_price = info.get('previousClose')


    last_sale = info.get("previousClose")
    fifty_two_week_high = info.get("fiftyTwoWeekHigh")
    fifty_two_week_low = info.get("fiftyTwoWeekLow")
    market_cap = info.get("marketCap")
    pe_ratio = info.get("trailingPE")
    open_price = info.get("open")  # Get the opening price for the day

    today = date.today()
    one_month_ago = today - timedelta(days=30)  # Approximate 1 month
    one_year_ago = today - timedelta(days=365)

    try:
        historical_data = stock.history(period="1y")  # Fetch 1 year of data
        month_ago_price = historical_data['Close'][one_month_ago]
        year_ago_price = historical_data['Close'][one_year_ago]

        change_1d = ((current_price - open_price) / open_price) * 100 if open_price != 0 else None # Calculate daily change
        change_1m = ((current_price - month_ago_price) / month_ago_price) * 100 if month_ago_price != 0 else None
        change_1y = ((current_price - year_ago_price) / year_ago_price) * 100 if year_ago_price != 0 else None

    except (IndexError, KeyError): # Handle cases where historical data might be incomplete
        change_1m = None
        change_1y = None
        change_1d = None

    return {
        "Company": info.get("shortName", ticker),
        "Ticker": ticker,
        "Bid": f"${info.get('bid'):.2f}" if info.get('bid') is not None else None,
        "Ask": f"${info.get('ask'):.2f}" if info.get('ask') is not None else None,
        "Current Price": f"${current_price:.2f}" if current_price is not None else None,
        "Last Sale": f"${last_sale:.2f}" if last_sale is not None else None,
        "Open": f"${info.get('open'):.2f}" if info.get('open') is not None else None,
        "High": f"${info.get('dayHigh'):.2f}" if info.get('dayHigh') is not None else None,
        "Low": f"${info.get('dayLow'):.2f}" if info.get('dayLow') is not None else None,
        "Market Cap (B)": f"${round(market_cap / 1e9, 2)}B" if market_cap is not None else None,
        "P/E Ratio": f"{pe_ratio:.2f}" if pe_ratio is not None else None,
        "Volume": info.get("volume"),
        "Avg. Volume": info.get("averageVolume"),
        "Exchange": info.get("exchange"),
        "Yield (%)": f"{round(info.get('dividendYield', 0) * 100, 2)}%" if info.get('dividendYield') is not None else None,
        "52W High": f"${fifty_two_week_high:.2f}" if fifty_two_week_high is not None else None,
        "52W Low": f"${fifty_two_week_low:.2f}" if fifty_two_week_low is not None else None,
        "1M Change": f"{change_1m:.2f}%" if change_1m is not None else None,
        "1Y Change": f"{change_1y:.2f}%" if change_1y is not None else None,
        "1D Change": f"{change_1d:.2f}%" if change_1d is not None else None, # Add daily change
    }

# ✅ Fetch Data for All Stocks
stock_data = [fetch_stock_data(ticker) for ticker in stock_symbols]

# ✅ Convert Data to Pandas DataFrame
df = pd.DataFrame(stock_data)

# ✅ Calculate % from 52-Week High (Trend Strength) and add the % sign to the column (and handle NaN values)
df["52W % from High"] = ((pd.to_numeric(df["Current Price"].str.replace('$', ''), errors='coerce') - pd.to_numeric(df["52W Low"].str.replace('$', ''), errors='coerce')) / (pd.to_numeric(df["52W High"].str.replace('$', ''), errors='coerce') - pd.to_numeric(df["52W Low"].str.replace('$', ''), errors='coerce'))) * 100
df["52W % from High"] = df["52W % from High"].apply(lambda x: f"{x:.2f}%" if pd.notna(x) else None)


# ✅ Sort by P/E Ratio (Lower is Usually Better)
df_sorted = df.sort_values(by=["P/E Ratio"], ascending=True)

# Display Results in Table Format (Left-Aligned)
print("\n📊 Stock Comparison Based on Key Metrics:\n")

# Apply CSS styling for left alignment:
display(HTML(df_sorted.to_html(classes='table-bordered', justify='left'))) # justify='left' is the key change


📊 Stock Comparison Based on Key Metrics:



,Company,Ticker,Bid,Ask,Current Price,Last Sale,Open,High,Low,Market Cap (B),P/E Ratio,Volume,Avg. Volume,Exchange,Yield (%),52W High,52W Low,1M Change,1Y Change,1D Change,52W % from High
0,Ally Financial Inc.,ALLY,$37.83,$38.70,$38.25,$38.25,$38.35,$38.87,$38.25,$11.7B,21.28,1950029,3295095,NYQ,3.14%,$45.46,$31.95,None,None,None,46.63%


In [25]:
# 📌 Install necessary libraries (Run this first in Google Colab)
!pip install yfinance pandas matplotlib --quiet

# ✅ Import Libraries
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML, display
from datetime import date, timedelta


# ✅ Define Stock Tickers to Compare (Modify this list as needed)
# stock_symbols = ["RY.TO", "TD.TO", "BMO.TO", "BNS.TO", "CM.TO", "SHOP.TO", "ENB.TO", "CNQ.TO", "CVE.TO", "MFC.TO", "WSHR.NE"]
# stock_symbols = ["WSHR.NE", "XGD.TO", "PLTR", "NVDA", "HDIF.TO"]  # Example tickers
stock_symbols = ["ALLY"]  # Example tickers

# # ✅ Print Definitions of Stock Metrics
# metric_definitions = {
#     "Bid": "The highest price a buyer is willing to pay for the stock.",
#     "Ask": "The lowest price a seller is willing to accept for the stock.",
#     "Last Sale": "The most recent price at which the stock was traded.",
#     "Open": "The stock's opening price for the trading session.",
#     "High": "The highest price reached during the session.",
#     "Low": "The lowest price reached during the session.",
#     "52W High": "The highest stock price recorded in the past 52 weeks.",
#     "52W Low": "The lowest stock price recorded in the past 52 weeks.",
#     "Market Cap (B)": "The total market value of the company's outstanding shares, measured in billions.",
#     "P/E Ratio": "The price-to-earnings ratio, which indicates how much investors are willing to pay for $1 of earnings.",
#     "Volume": "The total number of shares traded during the session.",
#     "Avg. Volume": "The average daily trading volume over a period.",
#     "Exchange": "The stock exchange where the stock is traded.",
#     "Yield (%)": "The dividend income as a percentage of the stock price.",
#     "Margin Requirement": "The percentage of the stock price that must be maintained as collateral if purchased on margin."
# }

# print("\n📚 Stock Metrics Definitions:\n")
# for metric, definition in metric_definitions.items():
#     print(f"🔹 {metric}: {definition}")

# ✅ Function to Fetch Stock Data from Yahoo Finance
def fetch_stock_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    # Get the current price (using 'regularMarketPrice' if available, otherwise 'previousClose')
    current_price = info.get('regularMarketPrice')
    if current_price is None:
        current_price = info.get('previousClose')

    last_sale = info.get("previousClose")
    fifty_two_week_high = info.get("fiftyTwoWeekHigh")
    fifty_two_week_low = info.get("fiftyTwoWeekLow")
    market_cap = info.get("marketCap")
    pe_ratio = info.get("trailingPE")

    today = date.today()
    one_month_ago = today - timedelta(days=30)  # Approximate 1 month
    one_year_ago = today - timedelta(days=365)

    try:
        historical_data = stock.history(period="1y")  # Fetch 1 year of data
        month_ago_price = historical_data['Close'][one_month_ago]
        year_ago_price = historical_data['Close'][one_year_ago]

        change_1m = ((current_price - month_ago_price) / month_ago_price) * 100 if month_ago_price != 0 else None
        change_1y = ((current_price - year_ago_price) / year_ago_price) * 100 if year_ago_price != 0 else None
    except (IndexError, KeyError): # Handle cases where historical data might be incomplete
        change_1m = None
        change_1y = None

    return {
        "Company": info.get("shortName", ticker),
        "Ticker": ticker,

        "Current Price": f"${current_price:.2f}" if current_price is not None else None,
        "1M Change": f"{change_1m:.2f}%" if change_1m is not None else None,
        "1Y Change": f"{change_1y:.2f}%" if change_1y is not None else None,

        "Bid": f"${info.get('bid'):.2f}" if info.get('bid') is not None else None,
        "Ask": f"${info.get('ask'):.2f}" if info.get('ask') is not None else None,

        "Last Sale": f"${last_sale:.2f}" if last_sale is not None else None,
        "Open": f"${info.get('open'):.2f}" if info.get('open') is not None else None,

        "High": f"${info.get('dayHigh'):.2f}" if info.get('dayHigh') is not None else None,
        "Low": f"${info.get('dayLow'):.2f}" if info.get('dayLow') is not None else None,

        "Market Cap (B)": f"${round(market_cap / 1e9, 2)}B" if market_cap is not None else None,
        "P/E Ratio": f"{pe_ratio:.2f}" if pe_ratio is not None else None,
        "Volume": info.get("volume"),
        "Avg. Volume": info.get("averageVolume"),

        "Exchange": info.get("exchange"),
        "Yield (%)": f"{round(info.get('dividendYield', 0) * 100, 2)}%" if info.get('dividendYield') is not None else None,

        "52W High": f"${fifty_two_week_high:.2f}" if fifty_two_week_high is not None else None,
        "52W Low": f"${fifty_two_week_low:.2f}" if fifty_two_week_low is not None else None,

    }

# ✅ Fetch Data for All Stocks
stock_data = [fetch_stock_data(ticker) for ticker in stock_symbols]

# ✅ Convert Data to Pandas DataFrame
df = pd.DataFrame(stock_data)

# ✅ Calculate % from 52-Week High (Trend Strength)
df["52W % from High"] = ((pd.to_numeric(df["Last Sale"].str.replace('$', ''), errors='coerce') - pd.to_numeric(df["52W Low"].str.replace('$', ''), errors='coerce')) / (pd.to_numeric(df["52W High"].str.replace('$', ''), errors='coerce') - pd.to_numeric(df["52W Low"].str.replace('$', ''), errors='coerce'))) * 100

# Add the % sign to the column (and handle NaN values)
df["52W % from High"] = df["52W % from High"].apply(lambda x: f"{x:.2f}%" if pd.notna(x) else None)


# ✅ Sort by P/E Ratio (Lower is Usually Better)
df_sorted = df.sort_values(by=["P/E Ratio"], ascending=True)

# Display Results in Table Format (Left-Aligned)
print("\n📊 Stock Comparison Based on Key Metrics:\n")

# Apply CSS styling for left alignment:
display(HTML(df_sorted.to_html(classes='table-bordered', justify='left'))) # justify='left' is the key change

# print("\n✅ Top Recommended Stocks Based on Analysis:\n")
# display(HTML(best_stocks.to_html(classes='table-bordered', justify='left'))) # justify='left' is the key change


# ✅ Identify Best Picks Based on Criteria (Value Investing)
# Convert P/E Ratio and Yield (%) to numeric (handle errors)
df_sorted["P/E Ratio"] = pd.to_numeric(df_sorted["P/E Ratio"].str.replace('$', '').str.replace('%', ''), errors='coerce')
df_sorted["Yield (%)"] = pd.to_numeric(df_sorted["Yield (%)"].str.replace('%', ''), errors='coerce')

# Convert "52W % from High" to numeric for filtering (handle errors)
df_sorted["52W % from High"] = pd.to_numeric(df_sorted["52W % from High"].str.replace('%', ''), errors='coerce')


# best_stocks = df_sorted[
#     (df_sorted["P/E Ratio"] < 20) &
#     (df_sorted["Yield (%)"] > 2) &
#     (df_sorted["52W % from High"] > 50)  # Now a numeric comparison
# ]


# # ✅ Plot P/E Ratio Comparison (Colab-Friendly)
# plt.figure(figsize=(20, 5))
# width = 0.5
# plt.bar(df["Company"], pd.to_numeric(df["P/E Ratio"].str.replace('$', ''), errors='coerce'), color='skyblue', width=width)
# plt.xlabel("Company")
# plt.ylabel("P/E Ratio")
# plt.title("P/E Ratio Comparison of Selected Stocks")
# plt.xticks(rotation=45, ha='right')
# plt.grid(axis="y", linestyle="--", alpha=0.7)
# plt.tight_layout()
# plt.show()


📊 Stock Comparison Based on Key Metrics:



,Company,Ticker,Current Price,1M Change,1Y Change,Bid,Ask,Last Sale,Open,High,Low,Market Cap (B),P/E Ratio,Volume,Avg. Volume,Exchange,Yield (%),52W High,52W Low,52W % from High
0,Ally Financial Inc.,ALLY,$38.25,None,None,$37.83,$38.70,$38.25,$38.35,$38.87,$38.25,$11.7B,21.28,1950029,3295095,NYQ,3.14%,$45.46,$31.95,46.63%
